In [98]:
import sklearn as sk
import sklearn.covariance as cov
import pandas as pd
import numpy as np
import ggplot as gg

In [99]:
import sklearn.preprocessing as pre
from DataFrameImputer import DataFrameImputer

In [100]:
# Import and replace NaNs with most frequent value for labels and mean for numericals
train_original = pd.DataFrame.from_csv("train.csv")
test_original = pd.DataFrame.from_csv("test.csv")
lenght_train = len(train_original)
total = train_original.append(test_original)
total['SalePrice'] = total['SalePrice'].fillna(value=0) # don't impute price of the test data

total_dframe = DataFrameImputer().fit_transform(total)

In [101]:
gg.ggplot(train_original, gg.aes(y='SalePrice',x='')) +  gg.geom_boxplot()

['Artery' 'Feedr' 'Norm' 'PosA' 'PosN' 'RRAe' 'RRAn' 'RRNe' 'RRNn'] ['Artery' 'Feedr' 'Norm' 'PosA' 'PosN' 'RRAe' 'RRAn' 'RRNn']


In [102]:
# Data Generation
loudness_zoning = {'A': 1,#Agriculture
       'C': 5,#Commercial
       'C (all)': 5, #Fuck-up            
       'FV': 3,#Floating Village Residential
       'I': 7,#Industrial
       'RH': 5,#Residential High Density
       'RL': 4,#Residential Low Density
       'RP': 2,#Residential Low Density Park 
       'RM': 5,#Residential Medium Density
                 }

loudness_condition = {
    'Artery':4 ,   #Adjacent to arterial street
       'Feedr':2 ,    # Adjacent to feeder street   
       'Norm':0 ,     #  Normal 
       'RRNn':2 ,     #Within 200' of North-South Railroad
       'RRAn':2 ,     #Adjacent to North-South Railroad
       'PosN':-2 ,     #Near positive off-site feature--park, greenbelt, etc.
       'PosA':-4 ,     #Adjacent to postive off-site feature
       'RRNe':2 ,     #Within 200' of East-West Railroad
       'RRAe':2 ,     #Adjacent to East-West Railroad
}

loudness_fence = {
     'GdPrv': 1,	#Good Privacy
       'MnPrv': 0,	#Minimum Privacy
       'GdWo': 0,	#Good Wood
       'MnWw': 0,	#Minimum Wood/Wire
       'NA': 0,	#No Fence
}

total_dframe['gen_loudness'] = total_dframe['Fence'].map(loudness_fence) + total_dframe['MSZoning'].map(loudness_zoning)+total_dframe['Condition1'].map(loudness_condition)+total_dframe['Condition2'].map(loudness_condition)

In [103]:
transportation_score = {
     'Artery': 2,   #Adjacent to arterial street
       'Feedr': 1,    # Adjacent to feeder street   
       'Norm': 0,     #  Normal 
       'RRNn': 1,     #Within 200' of North-South Railroad
       'RRAn': 2,     #Adjacent to North-South Railroad
       'PosN': 0,     #Near positive off-site feature--park, greenbelt, etc.
       'PosA': 0,     #Adjacent to postive off-site feature
       'RRNe': 1,     #Within 200' of East-West Railroad
       'RRAe': 2,     #Adjacent to East-West Railroad
}
total_dframe['gen_transport'] = total_dframe['Condition1'].map(transportation_score) + total_dframe['Condition2'].map(transportation_score)

In [104]:
type_score = {
        'WD': 0, #Warranty Deed - Conventional
       'CWD': 0,	#Warranty Deed - Cash
       'VWD': 0,	#Warranty Deed - VA Loan
       'New': 4,	#Home just constructed and sold
       'COD': 0,	#Court Officer Deed/Estate
       'Con': 2,	#Contract 15% Down payment regular terms
       'ConLw': 1,	#Contract Low Down payment and low interest
       'ConLI': 1,	#Contract Low Interest
       'ConLD': 1,	#Contract Low Down
       'Oth': 0,	#Other
}

condition_score = {
        'Normal': 0,	#Normal Sale
       'Abnorml': -1,	#Abnormal Sale -  trade, foreclosure, short sale
       'AdjLand':-1 ,	#Adjoining Land Purchase
       'Alloca': 0,	#Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       'Family': -1,	#Sale between family members
       'Partial': 2,	#Home was not completed when last assessed (associated with New Homes)
}

total_dframe['gen_discount'] = total_dframe['SaleType'].map(type_score) + total_dframe['SaleCondition'].map(condition_score)

In [105]:
park_score = {
     'Artery': -1,   #Adjacent to arterial street
       'Feedr': -1,    # Adjacent to feeder street   
       'Norm': 0,     #  Normal 
       'RRNn': 0,     #Within 200' of North-South Railroad
       'RRAn': -1,     #Adjacent to North-South Railroad
       'PosN': 1,     #Near positive off-site feature--park, greenbelt, etc.
       'PosA': 2,     #Adjacent to postive off-site feature
       'RRNe': 0,     #Within 200' of East-West Railroad
       'RRAe': -1,     #Adjacent to East-West Railroad
}

total_dframe['gen_parkness'] = total_dframe['Condition1'].map(park_score) + total_dframe['Condition2'].map(park_score)

In [106]:
total_dframe['gen_total_porch'] = total_dframe['OpenPorchSF']*0.7 + total_dframe['EnclosedPorch']+ total_dframe['3SsnPorch']*1.2+total_dframe['ScreenPorch']

In [107]:
def ascending(set, na=None):
    types = zip(set, range(1, len(set) + 1))

    if na != None:  # if there is a 'not appllicable' classification, generate an array with a 'has' and 'label' quality
        types = map(lambda x: [x[0], 1, x[1]], types)
        types.append([na, 0, 0])

    return dict(types)

In [108]:
# Label Properties
unneccessary = ['TotalBsmtSF', 'TotRmsAbvGrd']

special = ['MoSold', 'YrSold','SalePrice']

already_numerical = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                     'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
                     'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 
                     'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 
                     'GarageArea', 'WoodDeckSF',
                     'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal','OverallQual', 'OverallCond',
                    'gen_discount','gen_transport','gen_loudness', 'gen_total_porch', 'gen_parkness']

default = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
ordered = {
    'LandContour': ['Low', 'Lvl', 'Bnk', 'HLS'],
    'Utilities': ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'],
    'LandSlope': ['Gtl', 'Mod', 'Sev'],
    'ExterQual': default,
    'ExterCond': default,
    'BsmtQual': default,  # has an NA
    'BsmtCond': default,  # has an NA
    'BsmtExposure': ['Gd', 'Av', 'Mn', 'No'],  # has an NA
    'BsmtFinType1': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf'],  # NA
    'BsmtFinType2': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf'],
    'HeatingQC': default,
    'KitchenQual': default,
    'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
    'FireplaceQu': default,  # NA
    'GarageFinish': ['Fin', 'RFn', 'Unf'],  # NA
    'GarageQual': default,  # NA
    'GarageCond': default,  # NA
    'PavedDrive': ['Y', 'P', 'N'],
    'PoolQC': default,  # NA
    'Fence': ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'],
}

unordered = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LotConfig', 

             'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
             'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
             'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition'
             ]

In [109]:
# check for all colummns
assert set.intersection(set(already_numerical),set(ordered.keys())) == set()
assert set.intersection(set(unordered),set(ordered.keys())) == set()
assert set.intersection(set(unordered),set(already_numerical)) == set()

In [110]:
all_columns = set(unneccessary + special + already_numerical + list(ordered.keys()) + unordered)
assert all_columns.symmetric_difference(set(total_dframe.columns))==set()

In [111]:
data = pd.DataFrame()

data['SalePrice'] = total_dframe['SalePrice'] # make price the first entry
data['SellDate'] = total_dframe['YrSold']+total_dframe['MoSold']/12

In [112]:
for item in already_numerical:
    data[item] = total_dframe[item]

In [113]:
for item in ordered: # enumerate keys
    d = ascending(ordered[item])
    data[item] = list(map(lambda x:d[x],total_dframe[item]))

In [114]:
lable_binarizer = dict()
for item in unordered:
    lb = pre.LabelBinarizer()
    lable_binarizer[item] = lb #save for later decoding
    
    new_columns = lb.fit_transform(total_dframe[item])
    for class_,content in zip(lb.classes_, new_columns.T):
        data[item+" "+str(class_)] = content
        

In [115]:
train = data[0:lenght_train]
test = data[lenght_train:]

train.to_csv("cleaned_train.csv")
test.to_csv("cleaned_test.csv")